In [6]:
import pandas as pd

In [7]:
df_Renewable = pd.read_csv('../Dataset/한국전력거래소_인천시_제주_태양광_풍력_발전량_2024.csv', encoding='cp949')
df_climate = pd.read_csv('../Dataset/기후데이터_인천시_제주_2024.csv', encoding='cp949') 


/tmp/ipykernel_9760/2864723182.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_climate = pd.read_csv('../Dataset/기후데이터_인천시_제주_2024.csv', encoding='cp949')


In [9]:
df_climate.head()
df_Renewable.head()

,거래일자,거래시간,지역,연료원,전력거래량(MWh)
0,2024-01-01,1,인천시,태양광,0.0
1,2024-01-01,2,인천시,태양광,0.0
2,2024-01-01,3,인천시,태양광,0.0
3,2024-01-01,4,인천시,태양광,0.0
4,2024-01-01,5,인천시,태양광,0.0


In [11]:
import time
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1) 관측소 목록 (필요한 것만 예시로 넣었고, 전체를 여기에 확장하면 됩니다)
stations = [
    {"code": 112, "name": "인천"},
    {"code": 201, "name": "강화"},
    {"code": 102, "name": "백령도"},
    {"code": 98,  "name": "동두천"},
    {"code": 99,  "name": "파주"},
    {"code": 119, "name": "수원"},
    {"code": 202, "name": "양평"},
    {"code": 203, "name": "이천"},
    {"code": 184, "name": "제주"},
    {"code": 185, "name": "고산"},
    {"code": 188, "name": "성산"},
    {"code": 189, "name": "서귀포"},
]

# 2) 그룹(레이어) 정의: 필요에 맞게 확장
GROUP_SUDOKWON = {112, 201, 102, 98, 99, 119, 202, 203}   # 인천+경기(+백령도 포함 여부는 정책에 맞게)
GROUP_JEJU     = {184, 185, 188, 189}

df = pd.DataFrame(stations)

# 3) 지오코딩: 모호한 지명은 "검색어 보정"을 해주는 게 정확도가 올라갑니다.
#    (예: '고산', '성산'은 전국에 같은 이름이 있을 수 있어 제주로 명시)
query_override = {
    "제주":   "제주시, 제주특별자치도, South Korea",
    "고산":   "고산리, 한경면, 제주특별자치도, South Korea",
    "성산":   "성산읍, 서귀포시, 제주특별자치도, South Korea",
    "서귀포": "서귀포시, 제주특별자치도, South Korea",
    "강화":   "강화군, 인천광역시, South Korea",
    "백령도": "백령면, 옹진군, 인천광역시, South Korea",
}

geolocator = Nominatim(user_agent="kma_station_mapper")  # user_agent는 반드시 넣으세요.
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.1, swallow_exceptions=True)

def geocode_row(name: str):
    q = query_override.get(name, f"{name}, South Korea")
    loc = geocode(q)
    if loc is None:
        return None, None, q
    return loc.latitude, loc.longitude, q

lat_list, lon_list, q_list = [], [], []
for name in df["name"]:
    lat, lon, q = geocode_row(name)
    lat_list.append(lat)
    lon_list.append(lon)
    q_list.append(q)

df["lat"] = lat_list
df["lon"] = lon_list
df["geocode_query"] = q_list

# 4) 지오코딩 실패 데이터 확인(있으면 override 보강)
failed = df[df["lat"].isna() | df["lon"].isna()]
if not failed.empty:
    print("지오코딩 실패 항목(override 보강 필요):")
    print(failed[["code", "name", "geocode_query"]].to_string(index=False))

# 5) 지도 생성(중심점 = 평균 좌표)
valid = df.dropna(subset=["lat", "lon"]).copy()
center = [valid["lat"].mean(), valid["lon"].mean()]
m = folium.Map(location=center, zoom_start=7, tiles="OpenStreetMap")

# 6) 레이어(FeatureGroup) 구성
fg_sudokwon = folium.FeatureGroup(name="수도권(인천+경기)", show=True)
fg_jeju     = folium.FeatureGroup(name="제주", show=True)
fg_other    = folium.FeatureGroup(name="기타", show=False)

for _, r in valid.iterrows():
    popup = folium.Popup(f"Code: {int(r['code'])}<br>Name: {r['name']}", max_width=250)

    if int(r["code"]) in GROUP_SUDOKWON:
        folium.Marker([r["lat"], r["lon"]], popup=popup, icon=folium.Icon(color="blue")).add_to(fg_sudokwon)
    elif int(r["code"]) in GROUP_JEJU:
        folium.Marker([r["lat"], r["lon"]], popup=popup, icon=folium.Icon(color="green")).add_to(fg_jeju)
    else:
        folium.Marker([r["lat"], r["lon"]], popup=popup, icon=folium.Icon(color="gray")).add_to(fg_other)

fg_sudokwon.add_to(m)
fg_jeju.add_to(m)
fg_other.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# 7) 저장
out_path = "stations_map.html"
m.save(out_path)
print(f"저장 완료: {out_path}")


저장 완료: stations_map.html
